# Data entry inconsistencies

In [5]:
# environment setup
import pandas as pd
import numpy as np
import thefuzz
from thefuzz import process
import charset_normalizer

In [6]:
# read in all our data
professors = pd.read_csv("H:\\Datasets\\pakistan_intellectual_capital.csv")

In [7]:
professors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 13 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 1142 non-null   int64  
 1   S#                                         1142 non-null   int64  
 2   Teacher Name                               1142 non-null   object 
 3   University Currently Teaching              1142 non-null   object 
 4   Department                                 1142 non-null   object 
 5   Province University Located                1142 non-null   object 
 6   Designation                                1123 non-null   object 
 7   Terminal Degree                            1138 non-null   object 
 8   Graduated from                             1142 non-null   object 
 9   Country                                    1142 non-null   object 
 10  Year                    

In this file there are two columns of countries: professors and graduated from. Lets clean obth columns.

In [12]:
# let see professors countries
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()
countries = professors.Country.unique()
countries

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usofa', 'southkorea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'usa', 'finland', 'netherland',
       'greece', 'turkey', 'south korea', 'macau', 'singapore', 'spain',
       'japan', 'hongkong', 'saudi arabia', 'mauritius', 'scotland'],
      dtype=object)

with all in lower case and all stripped se see...
* south kirea instead of south korea
* usofa instead of usa


1. get the top 10 closest matches to "south korea"

In [19]:
# extract countries similar to south korea like southkorea
matches = process.extract("south korea", countries, limit=10)
matches

[('south korea', 100),
 ('southkorea', 95),
 ('uk', 57),
 ('norway', 54),
 ('ireland', 54),
 ('usofa', 51),
 ('usa', 45),
 ('austria', 44),
 ('saudi arabia', 43),
 ('scotland', 42)]

In [20]:
# build a function to replace any appearance of bad spelling of south korea
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = process.extract(string_to_match, strings, limit=10)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [21]:
# use the function to raplace the bad spelled south corea in dataframe profesors
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

All done!


## Examine another column : Graduated from

In [26]:
# let see professors countries
# convert to lower case
professors['Graduated from'] = professors['Graduated from'].str.lower()
# remove trailing white spaces
professors['Graduated from'] = professors['Graduated from'].str.strip()
countries = professors['Graduated from'].unique()
countries.sort()
countries
# check inconsitences like with "capital university of science & technology"

array(['abasyn university', 'abdul wali khan university, mardan',
       'abdus salam school of mathematical sciences,gc university',
       'agricultural university peshawar', 'allama iqbal open university',
       'asian institute of technology', 'aston university, birmingham',
       'australian national university, caneberra',
       'bahauddin zakariya university', 'bahria university',
       'bahria university,islamabad',
       'balochistan university of information technology, engineering and management sciences',
       'barani institute of information technology',
       'beaconhouse national university', 'beihang university',
       'beijing institute of technology',
       'beijing institute of technology beijing',
       'beijing university of posts & telecommunications',
       'biztek institute of business & technology,karachi',
       'blekinge institute of technology', 'boston university',
       'brock university canada', 'brunel university', 'bukc',
       'californi

Do the sema as before